** Fibonacci **

In [8]:
def Fib(n):
    if n <=2:
        return 1
    else:
        return Fib(n-1) + Fib(n-2)

In [10]:
Fib(10)

55

** Tower of Hanoi **

In [2]:
def moveDisk(fp, tp):
    print("moving disk from", fp, "to", tp)
    
def moveTower(height, fromPole, toPole, withPole):
    if height >= 1:
#         move top n-1 disks to middle pole, using toPole
        moveTower(height -1, fromPole, withPole, toPole)
#     move bottom disk
        moveDisk(fromPole, toPole)
#     move top n-1 disks from middle pole, using fromPole
        moveTower(height - 1, withPole, toPole, fromPole)

In [3]:
moveTower(3,"A","B","C")

moving disk from A to B
moving disk from A to C
moving disk from B to C
moving disk from A to B
moving disk from C to A
moving disk from C to B
moving disk from A to B


** Exploring a Maze **

** Dynamic Programming **

A classic example of an optimization problem involves making change using the fewest coins. Suppose you are a programmer for a vending machine manufacturer. Your company wants to streamline effort by giving out the fewest possible coins in change for each transaction. Suppose a customer puts in a dollar bill and purchases an item for 37 cents. What is the smallest number of coins you can use to make change? The answer is six coins: two quarters, one dime, and three pennies. How did we arrive at the answer of six coins? We start with the largest coin in our arsenal (a quarter) and use as many of those as possible, then we go to the next lowest coin value and use as many of those as possible. This first approach is called a greedy method because we try to solve as big a piece of the problem as possible right away.

If the amount does not match we have several options. 
What we want is the minimum of a penny plus the number of coins needed to make change for the ** original amount minus a penny**, 
or a nickel plus the number of coins needed to make change for the ** original amount minus five cents **, 
or a dime plus the number of coins needed to make change for the ** original amount minus ten cents ** , and so on. 
So the number of coins needed to make change for the original amount can be computed according to the following:

The algorithm for doing what we have just described is shown in Listing 7. In line 3 we are checking our base case; that is, we are trying to make change in the exact amount of one of our coins. If we do not have a coin equal to the amount of change, we make recursive calls for each different coin value less than the amount of change we are trying to make. Line 6 shows how we filter the list of coins to those less than the current value of change using a list comprehension. The recursive call also reduces the total amount of change we need to make by the value of the coin selected. The recursive call is made in line 7. Notice that on that same line we add 1 to our number of coins to account for the fact that we are using a coin. Just adding 1 is the same as if we had made a recursive call asking where we satisfy the base case condition immediately.

In [ ]:
def recMC(coinValueList,change):
    minCoins = change
    if change in coinValueList:
    return 1
    else:
        for i in [c for c in coinValueList if c <= change]:
            numCoins = 1 + recMC(coinValueList,change-i)
        if numCoins < minCoins:
            minCoins = numCoins
    return minCoins

print(recMC([1,5,10,25],63))

Each node in the graph corresponds to a call to recMC. The label on the node indicates the amount of change for which we are computing the number of coins. The label on the arrow indicates the coin that we just used. By following the graph we can see the combination of coins that got us to any point in the graph. The main problem is that we are re-doing too many calculations.

<img src = "http://interactivepython.org/runestone/static/pythonds/_images/callTree.png">

** The key to cutting down on the amount of work we do is to remember some of the past results so we can avoid recomputing results we already know. **

In [20]:
def recMC(coinValueList, change, resultList):
#     create a variable to record global optimal value
    minCoins = change
#     if change matches one of the coins
    if change in coinValueList:
#     create a list to store fathomed results (number of coins left)
#         it can be a dict/list with mapping of index(change amount): optimal result 
        resultList[change] = 1
        return 1
#     search the list first before recalculate for current change value
    elif resultList[change] > 0:
        return resultList[change]
    
    else:
        for i in [c for c in coinValueList if c <= change]:
#             reduce the case to searching the optimal solution for [change amount - coin value]
#             create a recursive call to original function with change - i
#             update the numCoins to account for i
            numCoins = 1 + recMC(coinValueList, change - i, resultList)
            if numCoins < minCoins:
                minCoins = numCoins
#                 update the resultList since this is result for a new change amount
                resultList[change] = numCoins
    return minCoins

In [25]:

print(recMC([1,5,10,25],63,[0]*64))

6


In [38]:
import time
%timeit recMC([1,5,10,21,25],73,[0]*74)

10000 loops, best of 3: 156 µs per loop


** The resultList needs to be maintained globally, and used as an input, since the function will be called recursively. **

In [13]:
def make_resultList(change):
    index = range(1, change+1)
    values = [0] * change
#     index_values = zip(index, values)
#     resultList = dict(index_values)
    return dict(zip(index, values))

In [33]:
def my_func():
    resultList = make_resultList(73)
    recMC([1,5,10,21,25],73, resultList)
    
%timeit my_func()

10000 loops, best of 3: 161 µs per loop


### REAL Dynamic Programming ###

** A truly dynamic programming algorithm will take a more systematic approach to the problem. Our dynamic programming solution is going to start with making change for one cent and systematically work its way up to the amount of change we require. This guarantees us that at each step of the algorithm we already know the minimum number of coins needed to make change for any smaller amount.
**

Let’s look at how we would fill in a table of minimum coins to use in making change for 11 cents. Figure 4 illustrates the process. We start with one cent. The only solution possible is one coin (a penny). The next row shows the minimum for one cent and two cents. Again, the only solution is two pennies. The fifth row is where things get interesting. Now we have two options to consider, five pennies or one nickel. How do we decide which is best? We consult the table and see that the number of coins needed to make change for four cents is four, plus one more penny to make five, equals five coins. Or we can look at zero cents plus one more nickel to make five cents equals 1 coin. Since the minimum of one and five is one we store 1 in the table. Fast forward again to the end of the table and consider 11 cents. Figure 5 shows the three options that we have to consider:

    1. A penny plus the minimum number of coins to make change for 11−1=1011−1=10 cents (1)
    2. A nickel plus the minimum number of coins to make change for 11−5=611−5=6 cents (2)
    3. A dime plus the minimum number of coins to make change for 11−10=111−10=1 cent (1)
    
    Either option 1 or 3 will give us a total of two coins which is the minimum number of coins for 11 cents.

<img src="http://interactivepython.org/runestone/static/pythonds/_images/changeTable.png">

When we have 11 cents to change, we have 3 coins to work with: 1) 1 cent 2) 5 cents 3) 10 cents. Using each will lead us back to one of the solutions we've already calculated.

The core idea is to grow the [solution list] to the cents we what to solve. Each time when we grow it, we take a look at what [coins] we can use and look up to solution of the remainder in the [solution list] and add 1. The min(solution_list[remainder] + 1) will give us the best solution for current amount of change.

<img src="http://interactivepython.org/runestone/static/pythonds/_images/elevenCents.png ">



In [36]:
def dpMakeChange(coinValueList,change,minCoins):
#     outer loop grows the solution list (minCoins)
    for cents in range(change+1):
#         set a large current value
        coinCount = cents
#         inner loop check the available coins can use
        for j in [c for c in coinValueList if c <= cents]:
#         then lookup the solution for the remainder (cents - coins_being_used)
#             the condition check if current coin provide optimal solution
            if minCoins[cents-j] + 1 < coinCount:
#         add one since using one additional coin
#                 update coincount to keep current best solution
                coinCount = minCoins[cents-j]+1
        minCoins[cents] = coinCount
    
    return minCoins[change]

In [37]:
%timeit dpMakeChange([1,5,10,21,25],73,[0]*74)

The slowest run took 4.93 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 76.5 µs per loop


In [40]:
def dpMakeChange(coinValueList,change,minCoins,coinsUsed):

    for cents in range(change+1):
        coinCount = cents
        newCoin = 1
        
        for j in [c for c in coinValueList if c <= cents]:
            if minCoins[cents-j] + 1 < coinCount:
                coinCount = minCoins[cents-j]+1
#             add one more variable to keep track of the coin being used when optimal solution is updated
                newCoin = j
    
        minCoins[cents] = coinCount
#         add a list to keep the new coin used in optimal solution for each change value
        coinsUsed[cents] = newCoin
    
    return minCoins[change]

In [41]:
def printCoins(coinsUsed,change):
    coin = change
    while coin > 0:
        thisCoin = coinsUsed[coin]
        print(thisCoin)
#         the remainder value of the change is the index of earlier optimal solution
        coin = coin - thisCoin

In [42]:
def main():
    amnt = 74
    clist = [1,5,10,21,25]
    coinsUsed = [0]*(amnt+1)
    coinCount = [0]*(amnt+1)

    print("Making change for",amnt,"requires")
    print(dpMakeChange(clist,amnt,coinCount,coinsUsed),"coins")
    print("They are:")
    printCoins(coinsUsed,amnt)
    print("The used list is as follows:")
    print(coinsUsed)

main()


Making change for 74 requires
5 coins
They are:
1
10
21
21
21
The used list is as follows:
[1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 21, 1, 1, 1, 25, 1, 1, 1, 1, 5, 10, 1, 1, 1, 10, 1, 1, 1, 1, 5, 10, 21, 1, 1, 10, 21, 1, 1, 1, 25, 1, 10, 1, 1, 5, 10, 1, 1, 1, 10, 1, 10, 21, 1, 5, 10, 21, 1, 1, 10, 21, 1, 10, 1]
